# Import modelflow

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from ipywidgets import interact,Dropdown,Checkbox,Tab
from IPython.display import display, clear_output,Latex, Markdown
import pandas as pd

import modeljupyter as mj
from modelvis import waterplot


from modelclass import model
model.widescreen()

import modelwidget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load model and data, and run 

In [ ]:
if not 'masia' in locals():
    print('Load model')
    masia,baseline = model.modelload('Asia.pcim',run=1,silent=1)
from inject import  inject
inject(masia)

In [ ]:
xx = masia(baseline,stats=0,progressbar=1)

In [ ]:
co2_input_var = [var for var in masia.totgraph.predecessors('WLD_CO2') if var.endswith('CO2')]

In [ ]:
co2_baseline = baseline.loc[:,co2_input_var].pipe(lambda df:df.rename(columns={c : masia.iso_dict.get(cname := c.split('_')[0],cname) for c in df.columns}))

# Variables to inputwidget

In [ ]:
outvar = 'GDI GCARBR CO2 LNN'.split()
displayvar = 'WLD_CO2 ASP_CO2 G20_CO2 '

#  masia.countries_GCARBR_A is list of all countries in the model with a countries_GCARBR_A 
#  masia.countries_ASEAN is list of all countries in ASEAN 
countries_rest = [country for country in masia.countries_GCARBR_A if country not in masia.countries_ASEAN]

slidedef_allasean =  {'All ASEAN countries' : {'var': ' '.join([f'{country}_GCARBR_A' for country in masia.countries_ASEAN ])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_allrest =  {'Rest of the world' : {'var': 
    ' '.join([f'{country}_GCARBR_A' for country in countries_rest])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_countries = {masia.iso_dict.get(country,country)+', Extra' : 
                      {'var': f'{country}_GCARBR_A', 'value': 0.0,'min':-40,'max':40,'op':'+impulse','step':10,'dec':5} 
                      for country in  masia.countries_ASEAN}



In [ ]:

from modelwidget import slidewidget, update_tab,update_input,tabwidget
from IPython.display import display

In [ ]:
atabwidget = update_tab({'Areas': {**slidedef_allasean,**slidedef_allrest},
                        'Country add on':slidedef_countries})

atabwidget.tabs

In [ ]:
testdf = baseline.copy()
atabwidget.update_df(testdf,masia.current_per)
zz = masia(testdf)

In [ ]:
masia.exodif().T

In [ ]:
this = update_input(masia,{'Area  wide carbon tax': {**slidedef_allasean,**slidedef_allrest},
                        'Single country add on':slidedef_countries})

this.wtotal

In [ ]:
updatevar = [f'{country}_GCARBR_A' for country in masia.countries_ASEAN ]
smalldf = baseline.loc[2022:2027, updatevar]

In [ ]:
smalldf.rename(columns= masia.get_a_country )

In [ ]:
xx = modelwidget.sheetwidget(df_var = smalldf,transpose=0,trans=masia.get_a_country)

xx.wsheet

In [ ]:
from modelwidget import slidewidget, tabwidget, sheetwidget, updatewidget

tabdef = {'Areawide Carbon tax 2022':slidewidget( {**slidedef_allasean,**slidedef_allrest}),
          'Single country add on 2022':slidewidget(slidedef_countries),
          'Individual tax rates' : sheetwidget(df_var = smalldf,transpose=0,trans=masia.get_a_country)
         }
wtab = tabwidget(tabdefdict = tabdef)
wtab.datawidget

In [ ]:
from modelwidget import slidewidget, tabwidget, sheetwidget, updatewidget

xx = updatewidget(masia,wtab)

xx.wtotal